In [ ]:
from esda.moran import Moran
import geopandas as gpd
import numpy as np
import pandas as pd
from libpysal.weights import Queen, KNN
from matplotlib import cm, colors
import matplotlib.pyplot as plt
from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap,truncate_colormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import multiprocessing as mp
from splot.esda import plot_moran
import statsmodels.api as sm
%env OMP_NUM_THREADS = 1

In [ ]:
df = pd.read_csv("./data/processed/Cleaned_Chicago_Sales.csv")
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
gdf.head()

In [ ]:
i_vars = [
    'Age',
    'Bedrooms',  
    'Building Square Feet',
    'Fireplaces', 
    'Full Baths',
    'Garage 1 Area',
    'Garage 2 Area', 
    'Half Baths', 
    'Land Square Feet',
    'Lot Size Squared',
    'Rooms'
    ]
dep_var = ['Sale Price']
X = df[i_vars]
y = df[dep_var]

In [ ]:
coords = [ np.array((geom.xy[0][0], geom.xy[1][0])) for geom in gdf['geometry'] ]
y, X = y.values, X.values

In [ ]:
THREADS = 2
pool = mp.Pool(THREADS)
gwr_selector = Sel_BW(coords, y, X)
gwr_bw = gwr_selector.search(pool=pool)

print("Selected optimal bandwidth is:", gwr_bw)

In [ ]:
gwr_results = GWR(coords, y, X, bw=gwr_bw).fit()

In [ ]:
gwr_results.summary()

In [ ]:
param_plots(gwr_results, gdf, names=["intercept"]+i_vars, subplots=(3,4), figsize=(10,20))